# 在"批踢踢實業坊(PTT)"網站抓股市看板上的新聞[[網站]](https://www.ptt.cc/bbs/Stock/index.html)

In [ ]:
# 可以去掉 python 輸出時，因為軟體版本所引起的警告的警告。
import warnings
warnings.filterwarnings('ignore')

**事前準備 : pip install pyquery**

In [ ]:
!pip install pyquery

#### PyQuery是一個類似於jQuery的解析網頁工具，使用lxml操作xml和html文檔，它的語法和jQuery很像。和XPATH，Beautiful Soup比起來，PyQuery更加靈活，提供增加節點的class信息，移除某個節點，提取文本信息等功能。
#### [PyQuery的用法](https://hiskio.com/courses/76/lectures/1885)

In [ ]:
from pyquery import PyQuery as pq
from collections import OrderedDict

import pandas as pd   # 當成 Python DataScience 的 Excel

In [ ]:
# 開使爬蟲動作
url = 'https://www.ptt.cc/bbs/Stock/index5003.html'  # index後的數字在PPT網站上隨時會更改，請找最新看板的前一頁(標示"上頁")的數字

q = pq(url)   # 直接對網站內容進行解析，並將結果傳回來

In [ ]:
# 經過解析後的 q 就是網站的 html DOM Tree
print(q)

In [ ]:
# 獲取本頁所有標題 作者 推文數資訊
title = q('.title a').text()

author = q('.author').text()

push = q('.hl').text()

In [ ]:
print(title)

In [ ]:
print(author)

In [ ]:
print(push)

**很明顯這樣的資訊並不是我們要的，因為所有文章的訊息都擠在一起了，所以接下來我們要針對每一篇文章分別爬資訊**

In [ ]:
info = []

for article in q('.r-ent'):
    pass     # pass 在 python 迴圈中表示什麼都不做

print('如果沒有重新解析:', article)

print()

print('重新解析後:', pq(article))

In [ ]:
# 獲取每篇文章分別的標題 作者 推文數資訊
info = []

for article in q('.r-ent'):

    info_dict = OrderedDict()

    # 每一篇文章的資訊分別塞到字典裡頭
    article_q = pq(article) # 記得要重新用 pq再解析一次
    
    info_dict['title'] = article_q('.title a').text()
    
    info_dict['author'] = article_q('.author').text()
    
    info_dict['push'] = article_q('.hl').text()

    # 將字典存入list
    info.append(info_dict)

In [ ]:
# 將list轉成pandas dataframe
data = pd.DataFrame(info)

# 文章索引值由 1 開始
data.index += 1

# 存檔
data.to_csv('ptt_stock_info.csv', encoding = 'utf-8', index = None)

In [ ]:
data.head()

**現在已經有了每篇文章個別的標題 作者 推文數資訊，那如果想知道每篇文章的推文該怎麼做呢?**

In [ ]:
# 獲取每篇文章的url
url_list = []

for node in q('.title a'):

    url = 'https://www.ptt.cc' + str(pq(node).attr('href')) # 記得要重新用 pq再解析一次
    
    url_list.append(url)

In [ ]:
# 進入每篇文章的 url 獲得詳細資訊
page_list = []

for page in url_list:

    page_dict = OrderedDict()

    page_q = pq(page)
    
    page_dict['author'] = page_q('.article-metaline:nth-child(1) .article-meta-value').text()
    
    page_dict['push_content'] = page_q('.push-content').text()

    page_list.append(page_dict)

In [ ]:
page_data = pd.DataFrame(page_list)

# 文章索引值由 1 開始
page_data.index += 1

page_data.to_csv('ptt_stock_pageinfo.csv', encoding = 'utf-8', index = None)

In [ ]:
page_data.head()